In [5]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_excel('Updated_Dataset.xlsx')

# Encode categorical variables if any
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))  # Ensure strings are encoded correctly
    label_encoders[column] = le

# Ensure all data is numeric
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')  # Convert non-numeric to NaN

df = df.dropna()  # Remove rows with NaN values

# Split dataset
X = df.drop(columns=['Disease'])  # Assuming 'Disease' is the target column
y = df['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler and label encoders
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

# Train Machine Learning models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)
gb.fit(X_train, y_train)

# Save trained models
joblib.dump(rf, 'random_forest.pkl')
joblib.dump(gb, 'gradient_boosting.pkl')

# Train Deep Learning model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(set(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# Save deep learning model
model.save('deep_learning_model.h5')

# Predictions
rf_pred = rf.predict(X_test)
gb_pred = gb.predict(X_test)
dl_pred = np.argmax(model.predict(X_test), axis=1)

# Ensemble: Majority Voting
final_pred = np.array([np.bincount([rf_pred[i], gb_pred[i], dl_pred[i]]).argmax() for i in range(len(y_test))])

# Accuracy
ensemble_acc = accuracy_score(y_test, final_pred)
print("Ensemble Model Accuracy:", ensemble_acc)


Epoch 1/50
65/65 [==============================] - 0s 872us/step - loss: 0.7258 - accuracy: 0.8827
Epoch 2/50
65/65 [==============================] - 0s 824us/step - loss: 0.2888 - accuracy: 0.9221
Epoch 3/50
65/65 [==============================] - 0s 760us/step - loss: 0.2329 - accuracy: 0.9221
Epoch 4/50
65/65 [==============================] - 0s 778us/step - loss: 0.1975 - accuracy: 0.9279
Epoch 5/50
65/65 [==============================] - 0s 812us/step - loss: 0.1793 - accuracy: 0.9279
Epoch 6/50
65/65 [==============================] - 0s 782us/step - loss: 0.1537 - accuracy: 0.9375
Epoch 7/50
65/65 [==============================] - 0s 753us/step - loss: 0.1340 - accuracy: 0.9490
Epoch 8/50
65/65 [==============================] - 0s 758us/step - loss: 0.1182 - accuracy: 0.9577
Epoch 9/50
65/65 [==============================] - 0s 745us/step - loss: 0.1059 - accuracy: 0.9615
Epoch 10/50
65/65 [==============================] - 0s 769us/step - loss: 0.1076 - accuracy: 0.9663

c:\Users\WELCOME\anaconda3\envs\machinelearning\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
import joblib
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Disease'] = label_encoder.fit_transform(df['Disease'])

# Save label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load dataset
df = pd.read_excel("Updated_Dataset.xlsx")

# Verify correct target column name
target_column = 'Disease'  # Update based on actual dataset column name

# Remove leading/trailing spaces in column names
df.columns = df.columns.str.strip()

# Convert non-numeric columns to strings and strip spaces
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Encode target variable
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column])

# Drop non-numeric columns except the target
X = df.select_dtypes(include=[np.number]).drop(columns=[target_column])
y = df[target_column]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save model and scaler
joblib.dump(model, "respiratory_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Model training complete and saved.")

Accuracy: 0.9730769230769231
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.97      1.00      0.99       234
           5       0.95      0.95      0.95        20
           6       0.00      0.00      0.00         2

    accuracy                           0.97       260
   macro avg       0.32      0.33      0.32       260
weighted avg       0.95      0.97      0.96       260

Model training complete and saved.


c:\Users\WELCOME\anaconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\WELCOME\anaconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\WELCOME\anaconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [21]:

# Save model and scaler
joblib.dump(model, "respiratory_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Model training complete and saved.")

Model training complete and saved.


In [22]:
def load_and_predict(input_data):
    loaded_model = joblib.load("respiratory_model.pkl")
    loaded_scaler = joblib.load("scaler.pkl")
    loaded_label_encoder = joblib.load("label_encoder.pkl")
    
    input_array = np.array(input_data).reshape(1, -1)
    input_scaled = loaded_scaler.transform(input_array)
    prediction = loaded_model.predict(input_scaled)
    disease_name = loaded_label_encoder.inverse_transform([prediction[0]])[0]
    
    return disease_name

# Example prediction
example_input = [1.2, 3.4, 5.6, 7.8, 9.0]  # Adjust based on dataset features
print("Predicted Disease:", load_and_predict(example_input))


Predicted Disease: Normal


c:\Users\WELCOME\anaconda3\envs\machinelearning\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
